##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ταξινόμηση Εικόνας με Συνελικτικά Δίκτυα (TF)

<table class="tfo-notebook-buttons" align="left">

  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>

</table>

Αυτός ο οδηγός δείχνει πώς να ταξινομήσουμε γάτες ή σκύλους από αντίστοιχες εικόνες. Δημιουργεί έναν ταξινομητή εικόνας χρησιμοποιώντας ένα σειριακό  μοντέλο (`tf.keras.Sequential`) και φορτώνει τα δεδομένα χρησιμοποιώντας τη κλάση: `tf.keras.preprocessing.image.ImageDataGenerator`. Θα αποκτήσετε πρακτική εμπειρία για τις ακόλουθες έννοιες:

* Δημιουργία ροής εισαγωγής δεδομένων (data pipeline) χρησιμοποιώντας  για εκμάθηση αποθηκευμένα δεδομένα (με τη κλάση: `tf.keras.preprocessing.image.ImageDataGenerator`).
* Overfitting - Πώς να το αναγνωρίσετε και να το αποτρέψετε.
* Data augmentation και dropout.


O οδηγός ακολουθεί μια κλασσική σειρά εργασιών μηχανικής μάθησης:

1. Εξέταση και κατανόηση των δεδομένων
2. Ροή εισαγωγής δεδομένων
3. Κατασκευή μοντέλου
4. Εκπαίδευση μοντέλου
5. Δοκιμή μοντέλου
6. Βελτίωση μοντέλου και επανάληψη της διαδικασίας


## Εισαγωγή πακέτων



Ας ξεκινήσουμε εισάγοντας τα απαιτούμενα πακέτα. Το πακέτο`os`  χρησιμοποιείται για την ανάγνωση αρχείων και τη δομή του καταλόγου, το `NumPy`  για τη μετατροπή της  python λίστας σε numpy array και για την εκτέλεση πράξεων με πίνακες και το `matplotlib.pyplot` στα γραφήματα και στην εμφάνιση των εικόνων.

Εισαγωγή κλάσεων Tensorflow και Keras.

In [0]:
import tensorflow as tf

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

## Φόρτωση δεδομένων


Ξεκινάμε κατεβάζοντας τα δεδομένα. Εδώ χρησιμοποιούμε μια φιλτραρισμένη έκδοση των δεδομένων <a href="https://www.kaggle.com/c/dogs-vs-cats/data" target="_blank">Dogs vs Cats</a>  από το Kaggle. Κατεβάζουμε τα αρχεία και τα αποθηκεύουμε στον κατάλογο "/ tmp /".


In [0]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

Τα δεδομένα έχουν την ακόλουθη δομή:

<pre>
<b>cats_and_dogs_filtered</b>
|__ <b>train</b>
    |______ <b>cats</b>: [cat.0.jpg, cat.1.jpg, cat.2.jpg ....]
    |______ <b>dogs</b>: [dog.0.jpg, dog.1.jpg, dog.2.jpg ...]
|__ <b>validation</b>
    |______ <b>cats</b>: [cat.2000.jpg, cat.2001.jpg, cat.2002.jpg ....]
    |______ <b>dogs</b>: [dog.2000.jpg, dog.2001.jpg, dog.2002.jpg ...]
</pre>

Αφού εξαχθούν τα περιεχόμενα, εκχωρούμε μεταβλητές με την κατάλληλη διαδρομή αρχείου για τα σετ εκπαίδευσης και επικύρωσης.

In [0]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [0]:
train_cats_dir = os.path.join(train_dir, 'cats')  # directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')  # directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # directory with our validation dog pictures

### Κατανόηση Δεδομένων

Ας δούμε πόσες εικόνες γάτας και σκύλου βρίσκονται στους καταλόγους εκπαίδευσης και επικύρωσης:

In [0]:
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

In [0]:
print('total training cat images:', num_cats_tr)
print('total training dog images:', num_dogs_tr)

print('total validation cat images:', num_cats_val)
print('total validation dog images:', num_dogs_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

Για ευκολία, θέτουμε τις μεταβλητές που θα χρησιμοποιηθούν για την προ-επεξεργασία και την εκπαίδευση του δικτύου:

In [0]:
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

## Προετοιμασία δεδομένων

Διαμορφώνουμε τις εικόνες σε τανυστές (tensors) κινητής υποδιαστολής (floats) πριν την εισαγωγή τους στο δίκτυο:

1. Διάβασμα των εικόνων από τον δίσκο.
2. Αποκωδικοποιηση των εικόνων και μετατροπή σε σωστή μορφή RGBD.
3. Μετατροπή σε τανυστές κινητής υποδιαστολής.
4. Μετατροπή των τανυστών από τιμές μεταξύ 0 and 255 σε τιμές μεταξύ 0 και 1, καθώς στα νευρωνικά δίκτυα είναι καλύτερα να έχουμε μικρές τιμές εισόδου.

Όλες αυτές οι εργασίες μπορούν να γίνουν με τη κλάση `ImageDataGenerator` που παρέχεται από την `tf.keras`. 
Η κλάση μπορεί να διαβάσει εικόνες από το δίσκο και να τις προεπεξεργαστεί σε κατάλληλους τανυστές. Θα δημιουργήσει επίσης γεννήτριες που θα μετατρέπουν αυτές τις εικόνες σε παρτίδες (batches) τανυστών - χρήσιμες κατά την εκπαίδευση του δικτύου.

In [0]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

Μετά τον καθορισμό των γεννητριών για την  εκπαίδευση και επικύρωση εικόνων, η μέθοδος `flow_from_directory`  φορτώνει τις εικόνες από το δίσκο, κάνει rescaling και αλλάζει το μέγεθος των εικόνων στις απαιτούμενες διαστάσεις.

In [0]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

In [0]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

### Επόπτευση των εικόνων για εκπαίδευση

Βλέπουμε μερικές εικόνες εκπαίδευσης εξάγοντας μια παρτίδα εικόνων από τη γεννήτρια κατάρτισης - που είναι 32 εικόνες σε αυτό το παράδειγμα - και στη συνέχεια σχεδιάζουμε 5 από αυτές με την `matplotlib`.

In [0]:
sample_training_images, _ = next(train_data_gen)

Η  συνάρτηση `next` επιστρέφει μια παρτίδα από το σύνολο δεδομένων. Η τιμή που επιστρέφει η  συνάρτηση `next` έχει τη μορφή `(x_train, y_train)`, όπου x_train είναι τα χαρακτηριστικά εκπαίδευσης και y_train, οι ετικέτες της. Αγνοούμε τις ετικέτες για να απεικονίσουμε μόνο τις εικόνες εκπαίδευσης.

In [0]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [0]:
plotImages(sample_training_images[:5])

## Δημιουργία μοντέλου


Το μοντέλο αποτελείται από τρία συνελικτικά μπλοκ συνελεύσεων με ένα max pool επίπεδο σε κάθε ένα από αυτά. Στην κορυφή μπαίνει  ένα πλήρως συνδεδεμένο επίπεδο με 512 νευρώνες με ενεργοποίηση relu.

In [0]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

### Μετάφραση (Compiling) μοντέλου

Επιλέγουμε για τη βελτιστοποίηση (optimizer) *ADAM* και *binary cross entropy*. Για να βλέπουμε την ακρίβεια εκπαίδευσης και επικύρωσης σε κάθε εποχή, δίνουμε το  `metrics`. 

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Σύνοψη μοντέλου
Βλέπουμε όλα τα επίπεδα του δικτύου χρησιμοποιώντας τη  μέθοδο `summary` του μοντέλου:

In [0]:
model.summary()

### Εκπαίδευση του μοντέλου

Χρησιμοποιούμε τη μέθοδο `fit_generator` της κλάσης `ImageDataGenerator` για την εκπαίδευση του δικτύου.

In [0]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

### Εμφάνιση αποτελεσμάτων

Στη συνέχεια βλέπουμε τα αποτελέσματα αφού τελειώσει η εκπαίδευση στο δίκτυο

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Όπως μπορείτε να δείτε από τα γραφήματα, η ακρίβεια της εκπαίδευση και η ακρίβεια επικύρωσης αποκλίνουν σε μεγάλο βαθμό και το μοντέλο έχει  ακρίβεια μόνο περίπου **70%**  στα δεδομέννα επικύρωσης.

Ας δούμε τι πήγε στραβά και ας προσπαθήσουμε να αυξήσουμε τη συνολική απόδοση του μοντέλου.

## Overfitting

Στα παραπάνω γραφήματα, η ακρίβεια  εκπαίδευσης αυξάνεται γραμμικά με την πάροδο του χρόνου, ενώ η ακρίβεια επικύρωσης κολλάει περίπου στο 70% κατά τη διαδικασία εκπαίδευσης. Η διαφορά στην ακρίβεια μεταξύ της προπόνησης και  επικύρωσης είναι φανερή - και αποτελεί σημάδι *overfitting*.

Όταν έχουμε μικρό αριθμό δειγμάτων εκπαίδευσης, το μοντέλο μερικές φορές μαθαίνει από θορύβους ή ανεπιθύμητες λεπτομέρειες από τα δείγματα - σε βαθμό που επηρεάζει αρνητικά την απόδοση του μοντέλου σε νέα δείγματα. Αυτό το φαινόμενο είναι γνωστό ως overfitting. Αυτό σημαίνει ότι το μοντέλο θα δυσκολεύεται στη γενίκευση σε ένα νέο σύνολο δεδομένων.

Υπάρχουν πολλοί τρόποι για την καταπολέμηση του overfitting στη διαδικασία εκπαίδευσης. Σε αυτόν τον οδηγό, θα χρησιμοποιήσουμε *data augmentation* και *dropout*.

## Επαύξηση Δεδομένων - Data augmentation

Γενικά, οverfitting συμβαίνει όταν υπάρχει μικρός αριθμός δειγμάτων εκπαίδευσης. Ένας τρόπος για να διορθώσουμε αυτό το πρόβλημα είναι να αυξήσουμε το σύνολο δεδομένων έτσι ώστε να διαθέτει επαρκή αριθμό δειγμάτων εκπαίδευσης. Η αύξηση δεδομένων ακολουθεί την προσέγγιση της δημιουργίας περισσότερων δεδομένων από τα υπάρχοντα δείγματα εκπαίδευσης, με τυχαίους μετασχηματισμούς που κατασκευάζουν νέες παραπλήσιες εικόνες. Ο στόχος είναι το μοντέλο να μη δει ποτέ την ίδια εικόνα ξανά κατά τη διάρκεια της εκπαίδευσης. Αυτό βοηθά στην έκθεση του μοντέλου σε περισσότερες πτυχές των δεδομένων και οδηγεί σε καλύτερη γενίκευση.

Θα το κάνουμε εδώ στη `tf.keras` με χρήση της κλάσης `ImageDataGenerator`. Περνάμε στο σύνολο δεδομένων διαφορετικούς μετασχηματισμούς  που θα εφαρμοστούν κατά τη διάρκεια της εκπαίδευσης.

### Αύξηση και οπτικοποίηση δεδομένων

Ξεκινάμε εφαρμόζοντας τυχαία οριζόντια αναστροφή στο σύνολο δεδομένων και βλέπουμε πώς φαίνονται κάποιες από τις εικόνες μετά τον μετασχηματισμό.

### Εφαρμογή οριζόντιας αναστροφής

Περνάμε το `horizontal_flip` ως όρισμα στη κλάση `ImageDataGenerator` και θέτουμε `True` για να εφαρμόσει αυτή τη μέθοδο.

In [0]:
image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

In [0]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

Για να δούμε πως δουλεύει, ας πάρουμε πέντε φορές ένα δείγμα απο τις εικόνες εκπαίδευσης, ώστε η επαύξηση να εφαρμοστεί πέντε φορές στην ίδια εικόνα.

In [0]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [0]:
# Re-use the same custom plotting function defined and used
# above to visualize the training images
plotImages(augmented_images)

### Τυχαία περιστροφή εικόνας

Ας δούμε και μια άλλη μέθοδο, την περιστροφή, όπου θα εφαρμόσουμε τυχαία περιστροφή 45 μοιρών στις εικόνες εκπαίδευσης.

In [0]:
image_gen = ImageDataGenerator(rescale=1./255, rotation_range=45)

In [0]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [0]:
plotImages(augmented_images)

### Εφαρμογή μεγέθυνσης (zoom)

Προσθέτουμε μια τυχαία επαύξηση μεγέθυνσης μέχρι 50%.

In [0]:
# zoom_range from 0 - 1 where 1 = 100%.
image_gen = ImageDataGenerator(rescale=1./255, zoom_range=0.5) # 

In [0]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [0]:
plotImages(augmented_images)

### Όλα μαζί

Ας εφαρμόσουμε τώρα όλες τις προηγούμενες επαυξήσεις μαζί. Αλλαγή κλίμακας, 45 μοίρες περιστροφή, μετατόπιση πλάτους και ύψους, οριζόντια περιστροφή και μεγέθυνση στις εικόνες εκπαίδευσης

In [0]:
image_gen_train = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=45,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    horizontal_flip=True,
                    zoom_range=0.5
                    )

In [0]:
train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary')

Ας δούμε πως θα φαίνεται η ίδια εικόνα αν την περάσουμε πέντε φορές, με τις μεθόδους επαύξησης που βαλαμε.

In [0]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

### Δημιουργία γενήτριας δεδομένων επικύρωσης

Γενικά, εφαρμόζουμε επαύξηση δεδομένων μόνο στα δείγματα εκπαίδευσης. Οπότε στα δείγματα επικύρωσης κάνουμε μόνο αλλαγή κλίμακας και τις ενώνουμε σε παρτίδες με την `ImageDataGenerator`.

In [0]:
image_gen_val = ImageDataGenerator(rescale=1./255)

In [0]:
val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                 directory=validation_dir,
                                                 target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                 class_mode='binary')

## Παράλειψη - Dropout

Μια άλλη τεχνική για τη μείωση του overfitting είναι η εισαγωγή *παράλειψης (dropout)* στο δίκτυο. Είναι μια μορφή *κανονικοποίησης (regularization)* που αναγκάζει τα βάρη στο δίκτυο να λαμβάνουν μόνο μικρές τιμές, γεγονός που καθιστά την κατανομή τιμών βάρους πιο  ομαλή και το δίκτυο μπορεί να μειώσει το overfitting σε μικρά δείγματα.

Όταν θέτουμε παράλειψη σε ένα επίπεδο, ένας τυχαίος αριθμός μονάδων εξόδου μηδενίζεται στο τρέχον επίπεδο κατά την εκπαίδευση. Ως την τιμή εισόδου, το Dropout παίρνει έναν δεκαδικό αριθμό, πχ 0.1, 0.2, 0.4, κ.λπ. που σημαίνει την τυχαία κατάργηση 10%, 20% ή 40% των μονάδων εξόδου από το τρέχον επίπεδο.

Κατά την εφαρμογή 0.1 παράλειψης σε ένα συγκεκριμένο επίπεδο, τυχαία το 10% των μονάδων εξόδου σε κάθε εποχή εκπαίδευσης μηδενίζεται.

Ας δημιουργήσουμε μια αρχιτεκτονική δικτύου με αυτήν τη νέα δυνατότητα παράλειψης και ας την εφαρμόσουμε στα διάφορα συνελικτικά και πλήρως διασυνδεμένα.

## Δημιουργία νέου δικτύου με το Dropouts

Here, you apply dropout to first and last max pool layers. Applying dropout will randomly set 20% of the neurons to zero during each training epoch. This helps to avoid overfitting on the training dataset.
Εδώ, εφαρμόζουμε την εγκατάλειψη στα πρώτα και τα τελευταία ανώτατα επίπεδα max-pool. Θέτουμε τυχαία το 20% των νευρώνων σε μηδέν κατά τη διάρκεια κάθε εποχής εκπαίδευσης. Αυτό βοηθά στην αποφυγή του overfitting στο σύνολο δεδομένων εκπαίδευσης.

In [0]:
model_new = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', 
           input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

### Μετάφραση μοντέλου

Μετά την εισαγωγή των παραλείψεων στο δίκτυο, ας κάνουμε μετάφραση του μοντέλου και προβολή της περίληψης επιπέδων.

In [0]:
model_new.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

model_new.summary()

### Εκπαίδευση μοντέλοη

Αφού εισάγουμε επαύξηση δεδομένων στα δείγματα εκπαίδευσης και προσθέσουμε παραλήψεις στο δίκτυο, ας εκπαιδεύσουμε το νέο δίκτυο:

In [0]:
history = model_new.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

### Εμφάνιση καμπυλών εκπαίδευσης

Βλέποντας τις καμπύλες ακρίβειας και σφάλματος στο νέο μοντέλο μετά την προπόνηση, βλέπουμε ότι υπάρχει σημαντικά λιγότερο overfitting από ό,τι πριν. Η ακρίβεια θα αυξηθεί μετά απο προπόνηση του μοντέλου σε περισσότερες εποχές.

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Συνέχεια εξερεύνησης...

Υπάρχουν και άλλες τεχνικές για μείωση του overfitting.
Δοκιμάστε τα εξής

*   *Μείωση της πολυπλοκότητας του μοντέλου*. Ποιά είναι η συμπεριφορά του αρχικού μοντέλου αν αφαιρεθεί πχ ένα συνελικτικό επίπεδο;
*   *Εφαρμογή L2 regularization*. Ποιά είναι η συμπεριφορά του αρχικού μοντέλου αν προστεθεί η [L2](https://www.tensorflow.org/api_docs/python/tf/keras/regularizers/l2) στα συνελικτικά επίπεδα;
*   [*Batch Normalization*](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization). Ποιά είναι η συμπεριφορά του αρχικού μοντέλου μετά την προσθήκη της;
*   [Global average pooling](https://codelabs.developers.google.com/codelabs/keras-flowers-convnets/#5). Αντί για χρήση πυκνών επιπέδων στο τέλος του δικτύου, για μείωση της πολυπλοκότητας του μοντέλου

Χωριστείτε σε ομάδες και δοκιμάστε αυτές τις μεθόδους. Στο τέλος συγκρίνετε τα αποτελέσματα! 







